<a href="https://colab.research.google.com/github/JulianSchmidtke/CAPTUM/blob/main/AsuD_CAPTUM_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip installs. Wenn Schon installiert, dann einfach nicht ausführen
!pip install networkx

!pip install spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

!pip install wordcloud

!pip install -q flair
!pip install -q numpy
!pip install -q pandas
!pip install -q gensim
!pip install -q hdbscan
!pip install -q bertopic
!pip install -q tensorflow
!pip install -q tensorflow_hub
!pip install -q tensorflow_text
!pip install -q torch
!pip install -q sentence_transformers

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nltk, os
import pandas as pd
import numpy as np
import re
from collections import Counter
import shutil

In [ ]:
# os.chdir(r'/content')

# content_dir = r'/content/CAPTUM/'

# # Remove if files already exist
# while os.path.exists(content_dir): 
#     shutil.rmtree(content_dir)


!git clone https://github.com/JulianSchmidtke/CAPTUM.git

# Duplikate unter den Papern entfernen anhand der Hash-Werte

In [ ]:
#Arbeitsverzeichnis auf das Ordnerverzeichnis ändern, in welchem die Daten liegen (txt-Format)
file_dir = r'./CAPTUM/files'
os.chdir(file_dir)

#Speichern der Dateinamen in einem Array
files = os.listdir()
df_files = pd.DataFrame(files, columns=['Filepath'])

In [ ]:
# create checksum for each file to identify duplicates
# https://stackoverflow.com/questions/16874598/how-do-i-calculate-the-md5-checksum-of-a-file-in-python#16876405
import hashlib
def get_checksum(filepath: str) -> str:
    # Open,close, read file and calculate MD5 on its contents 
    with open(filepath, 'rb') as file_to_check:
        # read contents of the file
        data = file_to_check.read()    
        # pipe contents of the file through
        return hashlib.md5(data).hexdigest()
df_files['Checksum'] = df_files['Filepath'].apply(get_checksum)

In [ ]:
fileNameTags = list(map(lambda fn: fn.replace('', '+').split('_')[0:-2],df_files['Filepath']))


df_files['Name'] = [fNT[-1] for fNT in fileNameTags]
df_files['Tags'] = [fNT[1:-1] for fNT in fileNameTags]
df_files['Year'] = [fY[-8:-4] for fY in df_files['Name']]
df_files['Authors'] = [fN.split(' ')[0] for fN in df_files['Name']]

In [ ]:
#Den Inhalt der Paper einlesen und in einen Pandas-
df_files['Content'] = df_files['Filepath'].apply(lambda f: open(f, 'r', encoding = "ISO-8859-1").read())
df_files

In [ ]:
checksums = df_files['Checksum'].unique()
for checksum in checksums:
    tags = df_files.loc[df_files['Checksum'] == checksum, 'Tags']
    list_of_pair_of_tags = tags.values.ravel()
    list_of_tags = [tag for tags in list_of_pair_of_tags for tag in tags]
    unique_tags = pd.unique(list_of_tags)
    df_files.loc[df_files['Checksum'] == checksum, 'Tags'] = ', '.join(map(str, unique_tags))

In [ ]:
# create dataframe for further analytics w/o the duplicate articles
df = df_files.drop_duplicates(subset=['Checksum'], keep='first')
df.reset_index(inplace=True, drop=True)
df = df.loc[:,['Authors','Year','Tags', 'Content']]

# Korpus erstellen

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

Es werden Füllwörter entfernt.

In [ ]:
stop_words=set(stopwords.words("english"))
def stoppingFunc(text, stop_words=stop_words):
    filtered_text=[]
    for word in text:
        # Wort unter 3 Zeichen -> weg
        if len(word) < 3:
          next
        # Mehr als eine Ziffer -> weg
        elif sum(c.isdigit() for c in word)>1:
          next
        # nicht in Stopword Liste -> behalten
        elif word not in stop_words:
          filtered_text.append(word)
        
    return filtered_text

Wörter auf ihren Wortstamm reduzieren (cats -> cat, transportation -> transport)

In [ ]:
def stemmingFunc(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in text]

Komplexere Version des Stemming. Wörter werden im Zusammenhang gesehen und reduziert. Verben werden auf den Infiitiv gesetzt usw.

In [ ]:
def lemmatizingFunc(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in text]

In [ ]:
def removeWords(text):
  filtered_text=[]
  for word in text:
    if word.lower() in ('csu', 'urticaria', 'doi', 'http', 'the', 'al', 'fig', 
                        'point', 'data', 'patient', 'result', 'study', 'disease', 
                        'result', 'group', 'table', 'figure', 'patient', 'antihistamine',
                        'cell', 'symptom'):
      next
    else:
      filtered_text.append(word)

  counter = Counter(filtered_text)
  filtered_text = [key for key, val in counter.items() if val > 1]

  return filtered_text

In [ ]:
def remove_doi(text):
  doi_regex = r"10.\d{4,9}\/[-._;()\/:A-Z0-9]+$"
  new_text = re.sub(doi_regex, '', text, flags=re.IGNORECASE)
  return new_text

In [ ]:
removeWords(['csu', 'TEST', 'DOI', 'doi', 'al', 'DOI'])

In [ ]:
def normalize_text(tokenizer, text):
  # lowercase text
  text = str(text).lower()
  # remove non-UTF
  text = text.encode("utf-8", "ignore").decode()
  # remove doi
  text = remove_doi(text)

  tokenized_text = tokenizer.tokenize(text)

  tokenized_text = removeWords(tokenized_text)
  tokenized_text = stoppingFunc(tokenized_text)
  #tokenized_text = stemmingFunc(tokenized_text)
  tokenized_text = lemmatizingFunc(tokenized_text)

  text = " ".join(tokenized_text)
  return text

Ich glaube es sollte entweder eine Stemming oder eine Lemmatizing Funktion angewendet werden.

Ich glaube wir sollten erst Lemmatizen und dann Stopwords entfernen.

In [ ]:
regex_tokenizer = nltk.RegexpTokenizer("\w+")
df['Corpus'] = df['Content'].apply(lambda c: normalize_text(regex_tokenizer, c))

regex_tokenizer2 = nltk.RegexpTokenizer('\w+[\.,]{0,1}')
df['Corpus_w_Punctuation'] = df['Content'].apply(lambda c: normalize_text(regex_tokenizer2, c))

# df['New_Corpus'] = df['Content'].apply(lambda c: simple_tokenizer(c))

In [ ]:
# corpus_list = df['Content'].to_list()
# remove_word_in_every_corpus('', corpus_list)

In [ ]:
df

# Wordcloud


In [ ]:
from wordcloud import WordCloud

In [ ]:
words = ' '.join(list(df['Corpus']))
wordcloud = WordCloud(width=1000, height=1000, max_words=1000).generate(words)
wordcloud.to_file("../../first_review.png")

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Distinct Tags

In [ ]:
type(list(df['Tags']))
tags = set(', '.join(list(df['Tags'])).split(', '))
print(*tags, sep="\n")

# Algorithmen anwenden

### ScispaCy

In [ ]:
import spacy
import scispacy

from spacy import displacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from collections import OrderedDict, Counter
from pprint import pprint


In [ ]:

def display_entities(model, document):
  doc = nlp(document)
  displacy_image = displacy.render(doc, jupyter=True, style='ent')
  entity_and_label = set([(X.text, X.label_) for X in doc.ents])
  return displacy_image, entity_and_label

In [ ]:
#As an example print the first Corpus from the df with the recogniced entities.
nlp = spacy.load('en_ner_bionlp13cg_md')
entities = display_entities(nlp, df['Corpus_w_Punctuation'][0])

In [ ]:
entities_dataframe = pd.DataFrame(entities[1],columns=['Entity','Label'])
entities_dataframe

In [ ]:
def entity_linker(linker_name,document):
  """A function tha accepts the name of a scispacy knowledgebase and 
  documents and returns the entity link details"""
  
  doc = nlp(document)
  try: 
    entity = doc.ents[0]
  except IndexError:
    entity = 'NaN'
  entity_details = []
  entity_details.append(entity)
  try:
    for linker_ent in entity._.kb_ents:
      Concept_Id, Score = linker_ent
      entity_details.appen('Entity_matching_Score :{}'.format(Score))
      entity_details.append(linker.kb.cui_to_entity[linker_ent[linker_ent[0]]])
  except AttributeError:
    pass
  return entity_details

In [ ]:
nlp = spacy.load('en_ner_bionlp13cg_md')
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"}) #parameters are tunable, so it can be set to return more than 2 entity matches.
doc = df['Corpus_w_Punctuation'][0]
doc = nlp(doc)

entity = doc.ents[0]

linker = nlp.get_pipe("scispacy_linker")
for ent in entity._.kb_ents:
	print(linker.kb.cui_to_entity[ent[0]])


### Bag of Words
Quelle: [towardsdatascience](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24)

In [ ]:
import gensim
import numpy as np
np.random.seed(2021)
import nltk
nltk.download('wordnet')

In [ ]:
processed_docs = df['Corpus'].apply(lambda c: c.split(' '))

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

### **LDA** (LatentDirichletAllocation) 
Quelle: [Medium-Artikel zu LDA](https://towardsdatascience.com/nlp-with-lda-latent-dirichlet-allocation-and-text-clustering-to-improve-classification-97688c23d98)

In [ ]:
#Turn tags into a set for faster checking of whether a tag exists or not
type(list(df['Tags']))
unique_tags = set(', '.join(list(df['Tags'])).split(', '))
print(*unique_tags, sep="\n")


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA


def find_topics(text):
  count_vectorizer = CountVectorizer()
  count_data = count_vectorizer.fit_transform(list(text))


  # One topic that has an avg of two words because most questions had 1/2 tags
  number_topics = 5
  number_words = 20

  # Create and fit the LDA model
  lda = LDA(n_components=number_topics, n_jobs=-1)
  lda.fit(count_data)
  words = count_vectorizer.get_feature_names()

  #Get topics from model. They are represented as a list e.g. ['military','army']
  topics = [[words[i] for i in topic.argsort()[:-number_words - 1:-1]] for (topic_idx, topic) in enumerate(lda.components_)]
  topic_groups = topics

  topics = np.array(topics).ravel()
  distinct_topics = list(set(topics))

  #Only use topics for which a tag already exists
  existing_topics = set.intersection(set(topics),unique_tags)

  

  return (words, distinct_topics, topic_groups, existing_topics)

In [ ]:
words, distinct_topics, topic_groups, existing_topics = find_topics(df['Corpus_w_Punctuation'])

In [ ]:
sorted(distinct_topics)


### **BERTopic** 
Topic modeling mit BERTopic -> siehe: https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6

Copy and Paste aus: https://colab.research.google.com/drive/1iqs9Y5_zLI6R6mAwlnapcxcUbKjpv2CC?usp=sharing#scrollTo=MgMaef_uZT0T
bzw.: https://www.youtube.com/watch?v=TLPmlVeEf1k

In [ ]:
from bertopic import BERTopic

In [ ]:
#Ursprüngliche Idee einen BiomedNLP-PubMedBERT zu nutzen um das Embedding zu erstellen.
from flair.embeddings import TransformerDocumentEmbeddings

#@title Transformer für das Embedding in BERtopic
Transformer = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext" #@param {type:"string"}
LoadedTransformer = TransformerDocumentEmbeddings(Transformer)

In [ ]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings

glove_embedding = WordEmbeddings('crawl')
document_glove_embeddings = DocumentPoolEmbeddings([glove_embedding])

In [ ]:
topic_model = BERTopic(language="english", calculate_probabilities=True) # We need the probabilities to visualize
topics = topic_model.fit_transform(list(df['Corpus_w_Punctuation']))

In [ ]:
# Get the most frequent topics
topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
print(f"{outliers} documents have not been classified")
print(f"The other {topic_freq['Count'].sum() - outliers} documents are {topic_freq['Topic'].shape[0]-1} topics")

In [ ]:
topic_freq

In [ ]:
print(f"There are {topic_freq['Count'].iloc[1]} documents that are talking about topic ID {topic_freq['Topic'].iloc[1]}")

In [ ]:
topic_model.get_topic(topic_freq['Topic'].iloc[0])

In [ ]:
topic_model.visualize_topics()

### Top2Vec und Doc2Vec

Quelle: https://github.com/bhattbhavesh91/Top2Vec-Demo/blob/main/Top2Vec-Notebook.ipynb

In [ ]:
!pip install top2vec
import numpy as np
import pandas as pd
from copy import deepcopy
from top2vec import Top2Vec

In [ ]:
docs = list(df["Corpus"].values)
docs[:5] #print top 5 papers

In [ ]:
model = Top2Vec(docs, embedding_model='doc2vec')

In [ ]:
topic_count = model.get_num_topics()

In [ ]:
topic_words, word_scores, topic_nums = model.get_topics(topic_count)

In [ ]:
for topic in topic_nums:
    model.generate_topic_wordcloud(topic)

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
#Erstellen einer Liste aller Wörter die im Corpus vorkommen
corpus_collection_in_words = [corpus.split(' ') for corpus in df['Corpus']]
flat_corpus_words = set(flatten(corpus_collection_in_words))
flat_corpus_words

keywords = [tag.split(' ') for tag in tags] #basierend auf den distict Tags der Corpora, tags entspricht dem zu beginn definierten Set.
flat_keywords = set(flatten(keywords))

searchable_keywords = set([word.lower() for word in flat_keywords if word.lower() in flat_corpus_words]) #Behalten der Keywords / Tags die selbst im Corpus auftreten.

In [ ]:
searchable_keywords

In [ ]:
#@title Anzahl der ähnlichen Wörter pro gesuchtem Keyword
number_of_similar_words =  50 #@param {type:"integer"}

similarity_dict = []
not_learned_by_model = []

for word in searchable_keywords:
  try: 
    words, word_scores = model.similar_words(keywords=[word], keywords_neg=[], num_words=number_of_similar_words)
    similarity_dict.append({word:dict(zip(words,word_scores))})
    next
  except ValueError:
    not_learned_by_model.append(word)
print('The following words have not been learned by the model an thus are not part of the similarity search:')
print(not_learned_by_model)
    

In [ ]:
similarity_dict

### TFIDF Vertorizer

In [ ]:
#@title Dateiinhalt, Korpus oder Korpus mit Satzzeichen
src = 'Corpus' #@param {type:"string"}
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1)).fit(df[src].values)

In [ ]:
len(tfidf_vectorizer.vocabulary_)

In [ ]:
tfidf_vectors = tfidf_vectorizer.fit_transform(df['Corpus'].values)

In [ ]:
# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf_vectors[0]
df_tfidf_sample = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"]) 
df_tfidf_sample.sort_values(by=["tfidf"],ascending=False)

### K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

K-Means für mehrere k's berechnen und anschließend plotten

In [ ]:
Sum_of_squared_distances = []
silhouettes = []
max_range = 10
K = range(2,max_range + 1)
for k in K: 
    print(str(k) + "/" + str(max_range))
    km = KMeans(n_clusters=k)
    km_labels = km.fit(tfidf_vectors)
    Sum_of_squared_distances.append(km_labels.inertia_)
    km_labels_2 = km.fit_predict(tfidf_vectors)
    silhouette_avg = silhouette_score(tfidf_vectors, km_labels_2)
    silhouettes.append(silhouette_avg)


In [ ]:
plt.plot(K,Sum_of_squared_distances, 'b^-')
plt.xlabel('k')
plt.ylabel('Summe der quadrierten Abweichungen')
plt.title('Elbow-Methode (Korpus mit Zeichensetzung)')
plt.show()

In [ ]:
plt.plot(K,silhouettes, 'b^-')
plt.xlabel('k')
plt.ylabel('Silouette-Score')
plt.title('Silouette-Score je Anzahl der Cluster')
plt.show()

Wortsammlungen erstellen durch angabe der Idealen Clusterzahl im Formularfeld

In [ ]:
#@title Anzahl der Cluster gem. Elbow-Curve
k = 8#@param {type:"integer"}

km = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1,
                verbose=1)

In [ ]:
#https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html
km = km.fit(tfidf_vectors)
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()

clustered_words = []
for i in range(k):
  words_per_cluster = []
  for ind in order_centroids[i, :5]:
    words_per_cluster.append(terms[ind])
  clustered_words.append(words_per_cluster)

In [ ]:
#clustered_words Liste transponieren
clustered_words = list(map(list, zip(*clustered_words)))

#Dataframe aus clustered_words erstellen
cluster_names = ['Cluster %s' %i for i in range(1,k+1)]
df_clustered_words = pd.DataFrame(data=clustered_words, columns=cluster_names)
df_clustered_words

#### Visualisierung

In [ ]:
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

G = nx.Graph()

words = []
color_map = []

for cluster in range(k):
  for row in clustered_words:
    if not G.has_node(row[cluster]):
      G.add_node(row[cluster], color=cluster)
      if cluster == 0:
          color_map.append('blue')
      elif cluster == 1: 
          color_map.append('green')
      elif cluster == 2: 
          color_map.append('red')
      elif cluster == 3: 
          color_map.append('yellow')
      elif cluster == 4: 
          color_map.append('grey')
      elif cluster == 5: 
          color_map.append('orange')
      elif cluster == 6: 
          color_map.append('black')
      elif cluster == 7: 
          color_map.append('brown')

    words.append(row[cluster])

corpus_list = df['Corpus'].to_list()

for word1 in words:
  for word2 in words:
    for corpus in corpus_list:
      if (word1 in corpus) and (word2 in corpus):
        if G.has_edge(word1, word2):
          G[word1][word2]['weight'] += 0.5
        elif G.has_edge(word2, word1):
          G[word2][word1]['weight'] += 0.5
        else:
          G.add_edge(word1, word2, weight=0.5)

pos = nx.spring_layout(G, scale=2)
nx.draw(G, pos=pos, with_labels=True, node_color=color_map)
nx.write_gexf(G, 'C:/temp/gephy.gexf')

### **HDBSCAN**

In [ ]:
#Es wurden die folgenden Transformer getestet:
# 1. allenai/scibert_scivocab_uncased
# 2. allenai/specter
# 3. microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext

#@title Transformer für Embedding
transformer_name = "allenai/specter" #@param {type:"string"}
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(transformer_name)
embeddings = model.encode(df['Corpus_w_Punctuation'], show_progress_bar=True)

In [ ]:
import umap
umap_embeddings = umap.UMAP(n_neighbors=12, 
                            n_components=2,
                            metric='cosine').fit_transform(embeddings)

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=5,
                          metric='euclidean',                      
                          cluster_selection_method='eom')
cluster = clusterer.fit(umap_embeddings)

In [ ]:
# Prepare data
umap_data = umap.UMAP(n_neighbors=12, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

In [ ]:
# Visualize clusters
fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BEBEBE', s=5)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=20, cmap=plt.cm.get_cmap('hsv_r', len(set(clustered.labels))))
plt.colorbar(ticks=range(len(set(clustered.labels))+1), label='Cluster')
plt.axis('off')
plt.clim(-0.5, (len(set(clustered.labels))+.5))

# KMeans in simpel am Stück

In [ ]:
stopWords = stopwords.words('english')

additional_stopwords = ['csu','chronic', 'spontaneous', 'urticaria', 'doi', 'http', 'the', 'al', 'fig', 
                        'point', 'data', 'patient', 'result', 'study', 'disease', 
                        'result', 'group', 'table', 'figure', 'patient', 'antihistamine',
                        'cell', 'symptom', 'crossref', 'pubm', 'activ', 'allergi', 'medi4212']

charfilter = re.compile('[a-zA-Z]{4,}')

def simple_tokenizer(text):
  text = remove_doi(text)
  text = text.lower()
  words = word_tokenize(text)
  words = [word for word in words if word not in stopWords]
  words = [word for word in words if word not in additional_stopwords]
  tokens = (list(map(lambda token: PorterStemmer().stem(token), words)))
  ntokens = list(filter(lambda token : charfilter.search(token),tokens))
  ntokens = [word for word in ntokens if word not in additional_stopwords]
  return ntokens

pca = list(df['Content'])

vec = TfidfVectorizer(tokenizer = simple_tokenizer, max_features = 1000, norm = 'l2', lowercase=True, ngram_range=(1,1))
pca_vec = vec.fit_transform(pca)

In [ ]:
# Tokenize Cluster again to compare
pca_tokenized = list(df['Content'].apply(lambda e: simple_tokenizer(e)))

In [ ]:
# KMeans berechnen für 20 Cluster
Sum_of_squared_distances_simple = []
silhouettes_simple = []
max_range = 20
K = range(2,max_range + 1)
for k in K: 
    print(str(k) + "/" + str(max_range))
    km = KMeans(n_clusters=k, max_iter=1000)
    km_labels = km.fit(pca_vec)
    Sum_of_squared_distances_simple.append(km_labels.inertia_)
    km_labels_2 = km.fit_predict(pca_vec)
    silhouette_avg = silhouette_score(pca_vec, km_labels_2)
    silhouettes_simple.append(silhouette_avg)

# Elbow für simples dataset
plt.plot(K,Sum_of_squared_distances_simple, 'b^-')
plt.xlabel('k')
plt.ylabel('Summe der quadrierten Abweichungen')
plt.title('Elbow-Methode')
plt.show()

# Silhouette für simples Dataset
plt.plot(K,silhouettes_simple, 'b^-')
plt.xlabel('k')
plt.ylabel('Silouette-Score')
plt.title('Silouette-Score je Anzahl der Cluster')
plt.show()

In [ ]:
# KMeans Cluster
k = 10
k_means = KMeans(k, max_iter=100)
clusters = k_means.fit_predict(pca_vec)

order_centroids = k_means.cluster_centers_.argsort()[:, ::-1]
terms = vec.get_feature_names()

clustered_words = []
for i in range(k):
  words_per_cluster = []
  for ind in order_centroids[i, :10]:
    words_per_cluster.append(terms[ind])
  clustered_words.append(words_per_cluster)

print(clustered_words)

In [ ]:
# Visualisierung
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout


colors = ['#dbc4fc','#5df243','#f594fc','#cea939','#585ff4','#e8dd7a','#bf4255',
'#ccf1ff','#4393c1','#ccf6ff','#c202d3','#a7a9e5','#d8adf4','#d35e10','#96a30d',
'#80fcf0','#eda6bc','#c66237','#ffb5b6','#fcfab5']

G = nx.Graph()

words = []
color_map = []

for i in range(len(clustered_words)):
  cluster = clustered_words[i]
  for word in cluster:
    if not G.has_node(word):
      G.add_node(word)
      color_map.append(colors[i])

    words.append(word)

for word1 in words:
  for word2 in words:
    for corpus in pca_tokenized:
      if (word1 in corpus) and (word2 in corpus):
        if G.has_edge(word1, word2):
          G[word1][word2]['weight'] += 0.5
        elif G.has_edge(word2, word1):
          G[word2][word1]['weight'] += 0.5
        else:
          G.add_edge(word1, word2, weight=0.5)

pos = nx.spring_layout(G, scale=2)
nx.draw(G, pos=pos, with_labels=True, node_color=color_map)